# notMNIST handwritten digits classification with MLPs

In this notebook, we'll train a multi-layer perceptron model to classify notMNIST digits using **Keras** (version $\ge$ 2 required). 

First, the needed imports. Keras tells us which backend (Theano, Tensorflow, CNTK) it will be using.

In [ ]:
%matplotlib inline

from pml_utils import get_notmnist, show_failures

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils import np_utils
from keras import backend as K

from distutils.version import LooseVersion as LV
from keras import __version__

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

print('Using Keras version:', __version__, 'backend:', K.backend())
assert(LV(__version__) >= LV("2.0.0"))

If we are using TensorFlow as the backend, we can check whether we have GPUs available:

In [ ]:
if K.backend() == "tensorflow":
    from tensorflow.python.client import device_lib
    print(device_lib.list_local_devices())

## notMNIST data set

Next we'll load the notMNIST data set.  First time we may have to download the data, which can take a while.


In [ ]:
#X_train, y_train, X_test, y_test = get_mnist('../MNIST', False, False)
import os
DATA_DIR = os.path.expanduser('~/data/notMNIST/')
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

X_train, y_train, X_test, y_test = get_notmnist(DATA_DIR)
nb_classes = 10

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# one-hot encoding:
Y_train = np_utils.to_categorical(y_train.view(np.int32)-ord('A'), nb_classes)
Y_test = np_utils.to_categorical(y_test.view(np.int32)-ord('A'), nb_classes)

print()
print('MNIST data loaded: train:',len(X_train),'test:',len(X_test))
print('X_train:', X_train.shape, X_train.dtype)
print('y_train:', y_train.shape, y_train.dtype)
print('Y_train:', Y_train.shape, Y_train.dtype)

The training data (`X_train`) is a matrix of size (529114, 784), i.e. it consists of 529114 letters expressed as 784 sized vectors (28x28 images flattened to 1D). `y_train` is a 529114-dimensional vector containing the correct class (i.e. one of: "A", "B", ..., "J") for each training letter.

Let's take a closer look. Here are the first 10 training letters:

In [ ]:
pltsize=1
plt.figure(figsize=(10*pltsize, pltsize))

for i in range(10):
    plt.subplot(1,10,i+1)
    plt.axis('off')
    plt.imshow(X_train[i,:].reshape(28, 28), cmap="gray")
    plt.title('Class: '+str(y_train[i]))
    print('Training sample',i,': class:',y_train[i], ', one-hot encoded:', Y_train[i])

## Multi-layer perceptron (MLP) network


### Initialization

Let's now create an MLP model that has multiple layers, non-linear activation functions, and dropout layers.  `Dropout()` randomly sets a fraction of inputs to zero during training, which is one approach to regularization and can sometimes help to prevent overfitting.

There are two options below, a simple and a bit more complex model.  Select either one.

The output of the last layer needs to be a softmaxed 10-dimensional vector to match the groundtruth (`Y_train`). 

Finally, we again `compile()` the model, this time using [*Adam*](https://keras.io/optimizers/#adam) as the optimizer.

In [ ]:
# Model initialization:
model = Sequential()

# A simple model:
model.add(Dense(units=20, input_dim=28*28))
model.add(Activation('relu'))

# A bit more complex model:
#model.add(Dense(units=50, input_dim=28*28))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))

#model.add(Dense(units=50))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))

# The last layer needs to be like this:
model.add(Dense(units=10, activation='softmax'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
print(model.summary())

In [ ]:
SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

### Learning

In [ ]:
%%time
epochs = 10 # one epoch with simple model takes about 4 seconds

history = model.fit(X_train.reshape((-1,28*28)), 
                    Y_train, 
                    epochs=epochs, 
                    batch_size=32,
                    verbose=2)

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['loss'])
plt.title('loss')

plt.figure(figsize=(5,3))
plt.plot(history.epoch,history.history['accuracy'])
plt.title('accuracy');

### Inference

Accuracy for test data.

In [ ]:
%%time
scores = model.evaluate(X_test.reshape((-1,28*28)), Y_test, verbose=2)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

We can take a closer look at the results, using the `show_failures()` function.

Here are the first 10 test letters the MLP classified to a wrong class:

In [ ]:
predictions = model.predict(X_test.reshape((-1,28*28)))

In [ ]:
predictions = np.argmax(predictions, axis=1)
predictions = [chr(x) for x in predictions+ord('A')]
predictions = np.array(predictions)
print(predictions[:10])

In [ ]:
show_failures(predictions, y_test, X_test)

We can use `show_failures()` to inspect failures in more detail. For example, here are failures in which the true class was "F":

In [ ]:
show_failures(predictions, y_test, X_test, trueclass='F')

## Model tuning

Modify the MLP model.  Try to improve the classification accuracy, or experiment with the effects of different parameters. 

You can also consult the Keras documentation at https://keras.io/.  For example, the Dense, Activation, and Dropout layers are described at https://keras.io/layers/core/.